# QUIC Connection Migration - Simple Tutorial

This notebook demonstrates QUIC connection migration step-by-step.

## Key Concept

**QUIC uses Connection IDs** (not IP addresses) to identify connections.
This allows the connection to survive network changes!

## Step 1: Install Dependencies

In [1]:
!pip install --upgrade aioquic colorlog

## Step 2: Import Libraries

In [2]:
import os
import time
from typing import List, Dict, Optional, Tuple
from dataclasses import dataclass
from datetime import datetime

print("✅ Imports successful!")

✅ Imports successful!


## Step 3: Understanding Connection IDs

Connection IDs are unique identifiers that stay the same even when IP addresses change.

In [3]:
def generate_connection_id(length: int = 8) -> bytes:
    """Generate a random connection ID"""
    return os.urandom(length)

# Generate a connection ID
conn_id = generate_connection_id(8)

print("Connection ID Analysis:")
print("=" * 50)
print(f"Length: {len(conn_id)} bytes")
print(f"Hexadecimal: {conn_id.hex()}")
print(f"Binary: {' '.join(format(b, '08b') for b in conn_id[:2])}... (first 2 bytes)")
print(f"Integer: {int.from_bytes(conn_id, 'big')}")
print("=" * 50)

print("\n💡 This ID will NEVER change during migration!")

Connection ID Analysis:
Length: 8 bytes
Hexadecimal: 0c0fbb41d21c651c
Binary: 00001100 00001111... (first 2 bytes)
Integer: 869119144478139676

💡 This ID will NEVER change during migration!


## Step 4: Create Packet Structure

In [4]:
@dataclass
class QuicPacket:
    """Represents a QUIC packet"""
    timestamp: float
    source_addr: Tuple[str, int]
    dest_addr: Tuple[str, int]
    connection_id: bytes
    packet_number: int
    packet_type: str
    payload: bytes
    
    def __str__(self):
        return f"""
  Timestamp: {datetime.fromtimestamp(self.timestamp).strftime('%H:%M:%S.%f')[:-3]}
  Source: {self.source_addr[0]:15}:{self.source_addr[1]}
  Dest:   {self.dest_addr[0]:15}:{self.dest_addr[1]}
  Conn ID: {self.connection_id.hex()[:16]}...
  Type: {self.packet_type:10} | Pkt #: {self.packet_number}
  Payload: {len(self.payload)} bytes
"""

print("✅ QuicPacket structure created")

✅ QuicPacket structure created


## Step 5: Create Simple QUIC Connection Simulator

In [5]:
class SimpleQuicConnection:
    """Simplified QUIC connection"""
    
    def __init__(self, role: str):
        self.role = role
        self.connection_id = generate_connection_id(8)
        self.local_addr = None
        self.peer_addr = None
        self.packet_number = 0
        self.packets: List[QuicPacket] = []
        
    def set_addresses(self, local: Tuple[str, int], peer: Tuple[str, int]):
        """Set local and peer addresses"""
        self.local_addr = local
        self.peer_addr = peer
        
    def send_packet(self, packet_type: str, payload: bytes) -> QuicPacket:
        """Send a QUIC packet"""
        self.packet_number += 1
        
        packet = QuicPacket(
            timestamp=time.time(),
            source_addr=self.local_addr,
            dest_addr=self.peer_addr,
            connection_id=self.connection_id,
            packet_number=self.packet_number,
            packet_type=packet_type,
            payload=payload
        )
        
        self.packets.append(packet)
        return packet
    
    def migrate_to(self, new_addr: Tuple[str, int]):
        """Migrate to new address"""
        old_addr = self.local_addr
        self.local_addr = new_addr
        return old_addr, new_addr

print("✅ SimpleQuicConnection created")

✅ SimpleQuicConnection created


## Step 6: ⭐ SIMULATE MIGRATION - WiFi to Cellular

This is the most important part! Watch carefully.

In [6]:
print("\n" + "="*70)
print("SCENARIO: Mobile Device Switches from WiFi to Cellular")
print("="*70 + "\n")

# Create client and server
client = SimpleQuicConnection("Client")
server = SimpleQuicConnection("Server")

# Initial setup on WiFi
print("📱 Step 1: Client connects on WiFi")
print("-" * 70)
client.set_addresses(
    local=("192.168.1.100", 50000),  # WiFi IP
    peer=("203.0.113.50", 443)
)
server.set_addresses(
    local=("203.0.113.50", 443),
    peer=("192.168.1.100", 50000)
)

print(f"Client IP: {client.local_addr[0]}")
print(f"Server IP: {server.local_addr[0]}")
print(f"Connection ID: {client.connection_id.hex()}")

# Send initial packets
print("\n🤝 Step 2: Establish connection")
print("-" * 70)
pkt1 = client.send_packet("INITIAL", b"Client Hello")
print(f"→ Client sends from: {pkt1.source_addr}")

pkt2 = server.send_packet("INITIAL", b"Server Hello")
print(f"← Server responds to: {pkt2.dest_addr}")

pkt3 = client.send_packet("HANDSHAKE", b"Finished")
print(f"→ Handshake complete")
print(f"✅ Connection established!")

# Send data on WiFi
print("\n📨 Step 3: Transfer data on WiFi")
print("-" * 70)
pkt4 = client.send_packet("DATA", b"Hello from WiFi!")
print(f"→ Packet #{pkt4.packet_number} sent from {pkt4.source_addr}")
print(f"  Connection ID: {pkt4.connection_id.hex()[:16]}...")

time.sleep(0.1)

# MIGRATION HAPPENS!
print("\n" + "="*70)
print("🔄 Step 4: NETWORK MIGRATION - WiFi → Cellular")
print("="*70)
print("User walks out of WiFi range, device switches to cellular...\n")

old_ip, new_ip = client.migrate_to(("10.20.30.40", 50001))
server.peer_addr = client.local_addr

print(f"  Old Address: {old_ip}  (WiFi)")
print(f"  New Address: {new_ip} (Cellular)")
print(f"\n  ⚠️  CONNECTION ID: {client.connection_id.hex()} (UNCHANGED!)")

# Path validation
print("\n🔍 Step 5: Server validates new path")
print("-" * 70)
print("Server must verify client really owns the new IP address")

challenge = os.urandom(8)
pkt5 = server.send_packet("CHALLENGE", b"PATH_CHALLENGE:" + challenge)
print(f"← Server sends PATH_CHALLENGE to: {pkt5.dest_addr}")
print(f"  Challenge data: {challenge.hex()}")

pkt6 = client.send_packet("RESPONSE", b"PATH_RESPONSE:" + challenge)
print(f"→ Client responds from: {pkt6.source_addr}")
print(f"  Response data: {challenge.hex()}")
print(f"  Connection ID: {pkt6.connection_id.hex()[:16]}... (SAME!)")
print("\n✅ Path validated!")

# Continue communication
print("\n📨 Step 6: Transfer continues on Cellular")
print("-" * 70)
pkt7 = client.send_packet("DATA", b"Hello from Cellular!")
print(f"→ Packet #{pkt7.packet_number} sent from {pkt7.source_addr}")
print(f"  Connection ID: {pkt7.connection_id.hex()[:16]}... (STILL SAME!)")

print("\n" + "="*70)
print("✨ MIGRATION COMPLETE!")
print("   • Connection NEVER dropped")
print("   • No re-handshake needed")
print("   • Seamless for application")
print("="*70)


SCENARIO: Mobile Device Switches from WiFi to Cellular

📱 Step 1: Client connects on WiFi
----------------------------------------------------------------------
Client IP: 192.168.1.100
Server IP: 203.0.113.50
Connection ID: 06711f46bc9c95e4

🤝 Step 2: Establish connection
----------------------------------------------------------------------
→ Client sends from: ('192.168.1.100', 50000)
← Server responds to: ('192.168.1.100', 50000)
→ Handshake complete
✅ Connection established!

📨 Step 3: Transfer data on WiFi
----------------------------------------------------------------------
→ Packet #3 sent from ('192.168.1.100', 50000)
  Connection ID: 06711f46bc9c95e4...



🔄 Step 4: NETWORK MIGRATION - WiFi → Cellular
User walks out of WiFi range, device switches to cellular...

  Old Address: ('192.168.1.100', 50000)  (WiFi)
  New Address: ('10.20.30.40', 50001) (Cellular)

  ⚠️  CONNECTION ID: 06711f46bc9c95e4 (UNCHANGED!)

🔍 Step 5: Server validates new path
----------------------------------------------------------------------
Server must verify client really owns the new IP address
← Server sends PATH_CHALLENGE to: ('10.20.30.40', 50001)
  Challenge data: 6e86735c3ae04d1f
→ Client responds from: ('10.20.30.40', 50001)
  Response data: 6e86735c3ae04d1f
  Connection ID: 06711f46bc9c95e4... (SAME!)

✅ Path validated!

📨 Step 6: Transfer continues on Cellular
----------------------------------------------------------------------
→ Packet #5 sent from ('10.20.30.40', 50001)
  Connection ID: 06711f46bc9c95e4... (STILL SAME!)

✨ MIGRATION COMPLETE!
   • Connection NEVER dropped
   • No re-handshake needed
   • Seamless for application


## Step 7: Inspect All Packets

Let's examine every packet that was sent.

In [7]:
print("\n" + "="*70)
print(f"ALL PACKETS (Total: {len(client.packets) + len(server.packets)})")
print("="*70)

all_packets = []
for pkt in client.packets:
    all_packets.append((pkt, "CLIENT"))
for pkt in server.packets:
    all_packets.append((pkt, "SERVER"))

all_packets.sort(key=lambda x: x[0].timestamp)

for pkt, sender in all_packets:
    print(f"\n[{sender}] Packet #{pkt.packet_number}:")
    print(pkt)
    print("-" * 70)


ALL PACKETS (Total: 7)

[CLIENT] Packet #1:

  Timestamp: 19:21:36.261
  Source: 192.168.1.100  :50000
  Dest:   203.0.113.50   :443
  Conn ID: 06711f46bc9c95e4...
  Type: INITIAL    | Pkt #: 1
  Payload: 12 bytes

----------------------------------------------------------------------

[SERVER] Packet #1:

  Timestamp: 19:21:36.261
  Source: 203.0.113.50   :443
  Dest:   192.168.1.100  :50000
  Conn ID: 524833a901a73c8b...
  Type: INITIAL    | Pkt #: 1
  Payload: 12 bytes

----------------------------------------------------------------------

[CLIENT] Packet #2:

  Timestamp: 19:21:36.261
  Source: 192.168.1.100  :50000
  Dest:   203.0.113.50   :443
  Conn ID: 06711f46bc9c95e4...
  Type: HANDSHAKE  | Pkt #: 2
  Payload: 8 bytes

----------------------------------------------------------------------

[CLIENT] Packet #3:

  Timestamp: 19:21:36.261
  Source: 192.168.1.100  :50000
  Dest:   203.0.113.50   :443
  Conn ID: 06711f46bc9c95e4...
  Type: DATA       | Pkt #: 3
  Payload: 16 byt

## Step 8: Timeline Visualization

In [8]:
print("\n" + "="*70)
print("TIMELINE OF EVENTS")
print("="*70 + "\n")

if all_packets:
    start_time = all_packets[0][0].timestamp
    
    prev_addr = None
    for pkt, sender in all_packets:
        elapsed_ms = (pkt.timestamp - start_time) * 1000
        
        # Detect migration
        if sender == "CLIENT" and prev_addr and prev_addr != pkt.source_addr:
            print(f"\n[+{elapsed_ms:6.1f}ms] 🔄 MIGRATION: {prev_addr} → {pkt.source_addr}\n")
        
        direction = "→" if sender == "CLIENT" else "←"
        print(f"[+{elapsed_ms:6.1f}ms] {direction} {pkt.packet_type:10} | "
              f"{pkt.source_addr[0]:15} → {pkt.dest_addr[0]:15}")
        
        if sender == "CLIENT":
            prev_addr = pkt.source_addr

print("\n" + "="*70)


TIMELINE OF EVENTS

[+   0.0ms] → INITIAL    | 192.168.1.100   → 203.0.113.50   
[+   0.0ms] ← INITIAL    | 203.0.113.50    → 192.168.1.100  
[+   0.1ms] → HANDSHAKE  | 192.168.1.100   → 203.0.113.50   
[+   0.1ms] → DATA       | 192.168.1.100   → 203.0.113.50   
[+ 101.4ms] ← CHALLENGE  | 203.0.113.50    → 10.20.30.40    

[+ 101.5ms] 🔄 MIGRATION: ('192.168.1.100', 50000) → ('10.20.30.40', 50001)

[+ 101.5ms] → RESPONSE   | 10.20.30.40     → 203.0.113.50   
[+ 101.9ms] → DATA       | 10.20.30.40     → 203.0.113.50   



## Step 9: TCP vs QUIC Comparison

In [9]:
print("\n" + "="*70)
print("TCP vs QUIC: How They Handle Network Switch")
print("="*70 + "\n")

print("🔴 TCP (Traditional):")
print("-" * 70)
print("Connection identified by: (IP1:port1, IP2:port2)")
print("")
print("When network changes:")
print("  1. IP address changes")
print("  2. ❌ 4-tuple no longer matches")
print("  3. ❌ Connection BREAKS")
print("  4. Must establish NEW connection")
print("  5. New TCP handshake (3-way)")
print("  6. New TLS handshake")
print("  7. Recreate application state")
print("")
print("  ⏱️  Total delay: ~300-500ms")
print("  😞 User sees: interruption, buffering, lag")

print("\n🟢 QUIC (Modern):")
print("-" * 70)
print(f"Connection identified by: {client.connection_id.hex()[:16]}...")
print("")
print("When network changes:")
print("  1. IP address changes")
print("  2. ✅ Connection ID stays the same")
print("  3. ✅ Connection SURVIVES")
print("  4. Send PATH_CHALLENGE")
print("  5. Receive PATH_RESPONSE")
print("  6. Continue communication")
print("")
print("  ⏱️  Total delay: ~50-100ms (1 RTT)")
print("  😊 User sees: seamless, no interruption")

print("\n" + "="*70)
print("💡 QUIC is 3-10x faster at handling network changes!")
print("="*70)


TCP vs QUIC: How They Handle Network Switch

🔴 TCP (Traditional):
----------------------------------------------------------------------
Connection identified by: (IP1:port1, IP2:port2)

When network changes:
  1. IP address changes
  2. ❌ 4-tuple no longer matches
  3. ❌ Connection BREAKS
  4. Must establish NEW connection
  5. New TCP handshake (3-way)
  6. New TLS handshake
  7. Recreate application state

  ⏱️  Total delay: ~300-500ms
  😞 User sees: interruption, buffering, lag

🟢 QUIC (Modern):
----------------------------------------------------------------------
Connection identified by: 06711f46bc9c95e4...

When network changes:
  1. IP address changes
  2. ✅ Connection ID stays the same
  3. ✅ Connection SURVIVES
  4. Send PATH_CHALLENGE
  5. Receive PATH_RESPONSE
  6. Continue communication

  ⏱️  Total delay: ~50-100ms (1 RTT)
  😊 User sees: seamless, no interruption

💡 QUIC is 3-10x faster at handling network changes!


## Step 10: Real-World Example - Video Download

Simulate downloading a video while switching networks.

In [10]:
print("\n" + "="*70)
print("SCENARIO: Downloading Video While Commuting")
print("="*70 + "\n")

# New client/server
video_client = SimpleQuicConnection("VideoClient")
video_server = SimpleQuicConnection("VideoServer")

# Start at home on WiFi
print("🏠 At home: Start video download on WiFi")
video_client.set_addresses(("192.168.1.50", 60000), ("203.0.113.10", 443))
video_server.set_addresses(("203.0.113.10", 443), ("192.168.1.50", 60000))

# Download chunks
print(f"  Connection ID: {video_client.connection_id.hex()[:16]}...")
video_client.send_packet("REQUEST", b"GET /video.mp4 chunk 1")
video_server.send_packet("DATA", b"X" * 1000)  # 1KB chunk
print("  ✅ Chunk 1/5 downloaded")

time.sleep(0.1)

video_client.send_packet("REQUEST", b"GET /video.mp4 chunk 2")
video_server.send_packet("DATA", b"X" * 1000)
print("  ✅ Chunk 2/5 downloaded")

time.sleep(0.1)

# Leave home, switch to cellular
print("\n🚗 Leaving home: Switch to cellular")
video_client.migrate_to(("10.5.5.5", 60001))
video_server.peer_addr = video_client.local_addr
print(f"  New IP: {video_client.local_addr[0]}")
print(f"  Connection ID: {video_client.connection_id.hex()[:16]}... (SAME!)")

# Continue download
video_client.send_packet("REQUEST", b"GET /video.mp4 chunk 3")
video_server.send_packet("DATA", b"X" * 1000)
print("  ✅ Chunk 3/5 downloaded")

time.sleep(0.1)

video_client.send_packet("REQUEST", b"GET /video.mp4 chunk 4")
video_server.send_packet("DATA", b"X" * 1000)
print("  ✅ Chunk 4/5 downloaded")

time.sleep(0.1)

# Arrive at office, switch to office WiFi
print("\n🏢 At office: Switch to office WiFi")
video_client.migrate_to(("172.16.1.100", 60002))
video_server.peer_addr = video_client.local_addr
print(f"  New IP: {video_client.local_addr[0]}")
print(f"  Connection ID: {video_client.connection_id.hex()[:16]}... (STILL SAME!)")

# Finish download
video_client.send_packet("REQUEST", b"GET /video.mp4 chunk 5")
video_server.send_packet("DATA", b"X" * 1000)
print("  ✅ Chunk 5/5 downloaded")

print("\n" + "="*70)
print("✨ Video download complete!")
print("   • Started on Home WiFi (192.168.1.50)")
print("   • Switched to Cellular (10.5.5.5)")
print("   • Finished on Office WiFi (172.16.1.100)")
print("   • SAME connection throughout!")
print("   • User never noticed the switches!")
print("="*70)


SCENARIO: Downloading Video While Commuting

🏠 At home: Start video download on WiFi
  Connection ID: 926c230816f6e32c...
  ✅ Chunk 1/5 downloaded


  ✅ Chunk 2/5 downloaded



🚗 Leaving home: Switch to cellular
  New IP: 10.5.5.5
  Connection ID: 926c230816f6e32c... (SAME!)
  ✅ Chunk 3/5 downloaded


  ✅ Chunk 4/5 downloaded

🏢 At office: Switch to office WiFi
  New IP: 172.16.1.100
  Connection ID: 926c230816f6e32c... (STILL SAME!)
  ✅ Chunk 5/5 downloaded

✨ Video download complete!
   • Started on Home WiFi (192.168.1.50)
   • Switched to Cellular (10.5.5.5)
   • Finished on Office WiFi (172.16.1.100)
   • SAME connection throughout!
   • User never noticed the switches!


## Summary

### What You Learned:

1. **Connection IDs** are the secret to QUIC migration
   - They don't change when network changes
   - TCP uses IP:port (which changes)

2. **Path Validation** ensures security
   - PATH_CHALLENGE/RESPONSE prevents spoofing
   - Only adds ~50-100ms

3. **Migration is seamless**
   - Connection survives network changes
   - No re-handshake needed
   - Application unaware of migration

4. **Real benefits**
   - Video streaming without buffering
   - Downloads survive network changes
   - Better mobile experience

### Try Next:

1. Open `http3_simulation.ipynb` for HTTP/3 over QUIC
2. Run `python quic_server.py` and `python quic_client.py`
3. Run `python migration_demo.py` for more scenarios
